In [104]:
import matplotlib.pyplot as plt
import torch
import tensorflow as tf 
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

In [105]:
#Importo i dati dalla libreria MNIST integrata in keras
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(f'X_train shape: {X_train.shape}, type: {type(X_train)}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (60000, 28, 28), type: <class 'numpy.ndarray'>
y_train shape: (60000,)
X_test shape: (10000, 28, 28)
y_test shape: (10000,)


In [106]:
# Create a TensorDataset object from the data
dataset = TensorDataset(torch.from_numpy(X_train, ).to(torch.float), torch.from_numpy(y_train).to(torch.float))
dataset_test = TensorDataset(torch.from_numpy(X_test, ).to(torch.float), torch.from_numpy(y_test).to(torch.float))
# Create a DataLoader object with batch size 32
batch_size = 600
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [107]:
class ConvNet(nn.Module):
    '''
    definition of a convolutional network in Pytorch
    '''
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels= 20, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels= 30, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=30, out_channels= 50, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=50 * 7 * 7, out_features=600)
        self.fc2 = nn.Linear(in_features=600, out_features=10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.relu(self.conv2(X))
        X = F.relu(self.conv3(X))
        X = torch.flatten(X, 1)
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        return X


In [108]:
device = torch.device('mps' if torch.mps.is_available() else 'cpu')
conv_nn = ConvNet().to(device)
print(f'The device used is: {device}')

The device used is: mps


In [109]:
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(conv_nn.parameters(), lr=learning_rate)

In [110]:
num_epochs = 50
n_total_steps = len(dataloader)
for epoch in range(num_epochs):

    running_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
#        print(type(images))
        images = images.reshape(batch_size, 1, 28, 28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = conv_nn(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.6f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(conv_nn.state_dict(), PATH)

[1] loss: 0.374345
[2] loss: 0.106332
[3] loss: 0.067450
[4] loss: 0.047710
[5] loss: 0.035680
[6] loss: 0.028083
[7] loss: 0.021298
[8] loss: 0.017351
[9] loss: 0.012741
[10] loss: 0.010492
[11] loss: 0.007082
[12] loss: 0.005642
[13] loss: 0.004326
[14] loss: 0.003098
[15] loss: 0.002930
[16] loss: 0.002193
[17] loss: 0.001417
[18] loss: 0.001157
[19] loss: 0.001124
[20] loss: 0.000870
[21] loss: 0.000694
[22] loss: 0.000626
[23] loss: 0.000513
[24] loss: 0.000428
[25] loss: 0.000400
[26] loss: 0.000341
[27] loss: 0.000303
[28] loss: 0.000267
[29] loss: 0.000252
[30] loss: 0.000233
[31] loss: 0.000207
[32] loss: 0.000194
[33] loss: 0.000172
[34] loss: 0.000161
[35] loss: 0.000148
[36] loss: 0.000134
[37] loss: 0.000124
[38] loss: 0.000113
[39] loss: 0.000105
[40] loss: 0.000094
[41] loss: 0.000088
[42] loss: 0.000082
[43] loss: 0.000075
[44] loss: 0.000070
[45] loss: 0.000066
[46] loss: 0.000061
[47] loss: 0.000057
[48] loss: 0.000053
[49] loss: 0.000048
[50] loss: 0.000044
Finished 

In [117]:
with torch.no_grad():
    n_correct = 0
    n_samples = len(dataloader_test.dataset)

    for (images, labels) in dataloader_test:
        images = images.reshape(images.shape[0], 1, 28, 28).to(device)
        labels = labels.to(device)

        outputs = conv_nn(images)

        # max returns (output_value ,index)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

Accuracy of the network on the 10000 test images: 98.83999999999999 %
